# This Notebook Inspects the Anisotropic Classifier

The anistotropic version of the classifier uses alternating horizontal and vertical 1D convolutions instead of 2D convolutions in the decoding phase of a segnet. 

- The notebook that modifies the _facade-segnet_ net is [modify-segnet-for-anisotropy.ipynb](anisotropic-training/modify-segnet-for-anisotropy.ipynb)
- I have made a notebook to help me visualize progress as I train, if is [train_using_jupyter.ipynb](train_using_jupyter.ipynb)
- After you have funished training, this notebook will allow you to inspect the net.

---
> **NOTE** When trianing I have my data in an external, larger hard drive. This gives me enough storage to hold many snapshots of the classifier as well as preprocessed imagery for traiing. I use a soft-link so that the extranl drive appears to be part of the project. 

---
> **NOTE:** Do not try to use more than one net at a time on the same GPU, use Kernel->Restart

In [ ]:
%pylab notebook

In order to use this we need a little of 4Gb of GPU RAM available
If some other process is using it, we need to stop them...

In [ ]:
!nvidia-smi
#!kill 10140

In [ ]:
cd ../..

In [ ]:
import os
sys.path.insert(0, os.path.abspath('./scripts/anisotropic-training'))

In [ ]:
NET_WEIGHTS = 'scripts/anisotropic-training/deploy/test_weights.caffemodel'
#NET_WEIGHTS = 'scripts/anisotropic-training/tn.caffemodel'
#NET_WEIGHTS='/home/shared/Projects/Facades/mybook/anisotropic/anisotropic_facades_iter_50000.caffemodel'
NET_LAYOUT = 'scripts/anisotropic-training/modified-inference-net.prototxt'

In [ ]:
import traceback

In [ ]:
import caffe
caffe.set_device(0)
caffe.set_mode_gpu()

In [ ]:
import warnings

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings(action='ignore')
    net = caffe.Net(NET_LAYOUT, NET_WEIGHTS, caffe.TEST)

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings(action='ignore')
    net2 = caffe.Net('scripts/anisotropic-training/modified-training-net.prototxt', NET_WEIGHTS, caffe.TEST)

In [ ]:
for param in net.params:
    for i in range(len(net.params[param])):
        a = net.params[param][i].data
        b = net.params[param][i].data
        print param,i, np.allclose(a,b)       

In [ ]:
net2.forward()

In [ ]:
data=repeat(net2.blobs['data'].data, 3, 0)
print data.shape

In [ ]:
labels = net.forward(data=data)

In [ ]:
from __future__ import division
x = 1
x/2

In [ ]:
def compare_activations(a1, a2, rows=10, cols=10, hpad=0.01):
    for r in range(rows):
        for c in range(cols):
            i = r*cols + c
            if i >= len(a1) or i >= len(a2):
                continue
            
            
            rleft = ( c/cols+hpad/2.,  (rows-r-1)/rows,     0.5/cols-hpad/2., 1./rows)
            rright = ((c+0.5)/cols,    (rows-r-1)/rows,     0.5/cols-hpad/2., 1./rows)
            left = axes(rleft)
            right = axes(rright)
            #print rleft
            
            eps = 0.001
            left.imshow((a1[i]-a1[i].mean())/(a1[i].std()+eps))
            left.set_xticks([]); left.set_yticks([]); left.set_xlabel('a{}'.format(i))
            right.imshow((a2[i]-a2[i].mean())/(a2[i].std()+eps))
            right.set_xticks([]); right.set_yticks([]); right.set_xlabel('b{}'.format(i))

In [ ]:
figure(figsize=(10,10))
compare_activations(net.blobs['conv1_1'].data[0], net2.blobs['conv1_1'].data[0])

In [ ]:
from glob import glob
files = glob('data/training/independant_12_layers/ours/npy/*.npy')

In [ ]:
index = random.choice(len(files))

In [ ]:
im = np.load(files[index])

In [ ]:
files[index]

In [ ]:
from pyfacades.util import channels_first, channels_last
figure()
imshow(channels_last(im[:3].astype(np.uint8)))

In [ ]:
NEGATIVE=0
POSITIVE=2
EDGE = 3

In [ ]:
from pyfacades.util import softmax

In [ ]:
figure()
index = random.choice(len(files))
im = np.load(files[index])
result = net.forward(blobs=['prob-window'], data=array([im[:3].copy(), im[:3].copy(), im[:3].copy()]))

subplot(121)
imshow(channels_last(im[:3])/255.)
xticks([]); yticks([])
xlabel('source')
subplot(122)
#imshow(net.blobs['prob-window'].data.mean(0)[POSITIVE],  vmin=0, vmax=1, cmap=cm.gray)
imshow(net.blobs['prob-window'].data[0][POSITIVE],  vmin=0, vmax=1, cmap=cm.gray)

xticks([]); yticks([]); xlabel('prob-windows')